## **Automated Optimal Angle Finder for Favored Mach Number**

In [1]:
import matplotlib.pyplot as plt
import csv
import math
import numpy as np
import pandas as pd
from itertools import product
import numpy as np
from scipy.optimize import minimize_scalar
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Functions

def xy(ethetas):
    thetas = np.cumsum(ethetas)
    xs = np.arange(0,len(ethetas))
    ys = np.repeat(-1.0,len(ethetas))
    ys[0] = 0
    tans = np.tan(thetas*np.pi/180)
    for i in range(1,len(ys)):
        updt = ys[i-1] + tans[i]
        ys[i] = updt

    return xs,ys

def thetas(x, y):
    theta = np.repeat(-1.0,len(x))
    theta[0] = 0.0
    for i in range(1,len(x)):
        s = np.sum(theta[:i])
        theta[i] = math.atan((y[i] - y[i-1]) / (x[i] - x[i-1]) ) * 180/math.pi
        theta[i] = theta[i] - s

    if np.any(np.array(theta) < 0):
        raise Exception("Negative angles.")
    return theta

def plot_inlet(angles,flip=True):
    xs,ys = xy(angles)
    fig = plt.figure()
    ax = fig.add_subplot()
    ys = np.max(ys)-ys
    for i in range(len(xs)):
        ax.plot(xs[i:(i+2)],ys[i:(i+2)]);

    ax.hlines(np.max(ys),0,np.max(xs)*1.25)
    ax.hlines(np.min(ys),np.max(xs),np.max(xs)*1.25)
    ax.hlines(np.min(ys),np.max(xs),np.max(xs)*1.25)
    ax.hlines(-0.1*(np.max(ys)-np.min(ys)),0.9*np.max(xs),np.max(xs)*1.25)

    ax.set_aspect('equal', adjustable='box')
    return fig, ax

def nu(M, gamma):
    return math.sqrt((gamma + 1) / (gamma - 1)) * math.atan(
        math.sqrt((gamma - 1) / (gamma + 1) * (M ** 2 - 1))) - math.atan(math.sqrt(M ** 2 - 1))

def expansion_mach(M1,theta,gamma):
    A = theta*math.pi/180+nu(M1,gamma)
    loss = lambda M2: (nu(M2,gamma)-A)**2
    res = minimize_scalar(loss, bounds=(1, 10), method='bounded')
    return(res.x)

def expansion_p(p1,M1,M2,gamma):
    top = 1+(gamma-1)/2*M1**2
    bottom = 1+(gamma-1)/2*M2**2
    r = (top/bottom)**(gamma/(gamma-1))
    return p1*r

def compression_beta(theta, mach, gamma):
    n = 0 # 0 = weak shock, 1 = strong shock
    theta = theta * math.pi/180.0;
    mu = math.asin(1/mach);
    c = math.tan(mu)**2;
    a = ((gamma-1)/2 + (gamma+1) * c/2) * math.tan(theta);
    b = ((gamma+1)/2 + (gamma+3) * c/2) * math.tan(theta);
    d = math.sqrt(4*(1-3*a*b)**3/((27*a**2*c+9*a*b-2)**2)-1);
    beta = math.atan((b+9*a*c)/(2*(1-3*a*b))-(d*(27*a**2*c+9*a*b-2))/(6*a*(1-3*a*b))*math.tan(n*math.pi/3+1/3*math.atan(1/d)))*180.0/math.pi
    return beta

def compression_mach(mach1, theta, beta, gamma):
    theta = theta * math.pi/180.0;
    beta = beta * math.pi/180.0;
    mach2 = (1/math.sin(beta - theta)) * math.sqrt((1 + 0.5*(gamma-1)*mach1**2*math.sin(beta)**2)/(gamma*mach1**2*math.sin(beta)**2 - 0.5*(gamma-1)))
    return mach2

def p_to_p_tot(p, mach, gamma):
    p_tot = p / ((1 + 0.5*(gamma-1)*mach**2)**(-1*gamma / (gamma-1)))
    return p_tot

def compression_p(mach1, p1, beta, gamma):
    beta = beta * math.pi/180.0
    p2 = p1*((2*gamma*mach1**2*math.sin(beta)**2 - (gamma-1)) / (gamma+1))
    return p2

def run_model(mach_inf,p_inf,angles,aoa=0,gamma=1.4,verbose=False):

    x,y = xy(angles)

    mach = []
    beta = []
    p = []
    p_tot = []

    # Initialize Region 0
    mach.append(mach_inf)
    beta.append(0)
    p.append(p_inf)
    p_tot.append(p_to_p_tot(p[0], mach[0], gamma))

    theta = thetas(x, y)
    effective_theta = theta#[theta[0], theta[1]+aoa, theta[2], theta[3], theta[4], theta[5]]
    effective_theta[1] = effective_theta[1] + aoa
    if np.sum(effective_theta[:-1])>=90:
        raise Exception("Sum of effective angles is bigger than 90.")


    # Solve for Regions 1 through 5
    for i in range(1, len(theta)):
        beta.append(compression_beta(effective_theta[i], mach[i-1], gamma))
        mach.append(compression_mach(mach[i-1], effective_theta[i], beta[i], gamma))
        p.append(compression_p(mach[i-1], p[i-1], beta[i], gamma))
        p_tot.append(p_to_p_tot(p[i], mach[i], gamma))

    if verbose:
        print(mach)
        print(beta)
        print(p)
        print(p_tot)

    mach_output = mach[len(mach)-1]
    p_tot_output = p_tot[len(p_tot)-1]
    p_tot_inf = p_inf/((1+mach_inf**2*(gamma-1)/2)**(-gamma/(gamma-1)))

    return {'mach':mach_output, 'pr':p_tot_output/p_tot_inf}

# Begin

L = np.linspace(1,10,10)
# H = np.linspace(1,10,10000)

pr_matrix = []
mach_matrix = []
master_matrix = []
identity_column = []

for i in L:
  row = []
  rowm = []
  for j in L:
    column = []
    columnm = []
    for k in L:
      vector = []
      vectorm = []
      for l in L:
        #4 Different Inputs, Add more if continuing this design
        et = np.array([0,i,j,k,l])
        result = run_model(mach_inf=5, p_inf=5532, angles=et)
        vector.append(result['pr'])
        vectorm.append(result['mach'])
        identity_column.append(et)
      row.append(vector)
      rowm.append(vectorm)
    column.append(row)
    columnm.append(rowm)
  pr_matrix.append(column)
  mach_matrix.append(columnm)

pr = np.array(pr_matrix).flatten()
m = np.array(mach_matrix).flatten()
id = np.vstack(identity_column)

master_matrix = np.column_stack((id, pr, m))

X = master_matrix[:, [5, 6]]
y = master_matrix[:, :5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(2,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(5)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=int(input("Epochs: ")), batch_size=32, validation_split=0.2)

# Calibrated test number to best fit output to 2.5 (model will tend to go under in order to increase pressure)
test_mach_number = float(input("Enter Desired Mach Number: "))

if -10 < test_mach_number < 10:
  X_test_mach = np.copy(X_test)
  X_test_mach[:, 1] = test_mach_number
  predicted_locations = model.predict(X_test_mach)

  sorted_indices = np.argsort(-X_test[:, 1])
  sorted_locations = predicted_locations[sorted_indices]

  top_locations = sorted_locations[:10]

  mach_vectors = []
  pressure_vectors = []

# Top 5 Angles, change if you want more
  for i in range(5):
      corrected_locations = top_locations[i, 1:].flatten()
      et = np.array([0] + list(corrected_locations))
      result = run_model(mach_inf=5, p_inf=5532, angles=et)
      mach_vectors.append(result['mach'])
      pressure_vectors.append(result['pr'])

  Five_or_Nine = None
  while Five_or_Nine not in ["5","9"]:
    Five_or_Nine = input("5 angles or 9 angles? (5/9): ")

    if Five_or_Nine in ["5","9"]:
      if Five_or_Nine == "5":
        print("Top 5:")
        print("1. Angles:", [0, top_locations[0,1:].tolist()], "Mach Speed:", mach_vectors[0], "Pressure:", pressure_vectors[0])
        print("2. Angles:", [0, top_locations[1,1:].tolist()], "Mach Speed:", mach_vectors[1], "Pressure:", pressure_vectors[1])
        print("3. Angles:", [0, top_locations[2,1:].tolist()], "Mach Speed:", mach_vectors[2], "Pressure:", pressure_vectors[2])
        print("4. Angles:", [0, top_locations[3,1:].tolist()], "Mach Speed:", mach_vectors[3], "Pressure:", pressure_vectors[3])
        print("5. Angles:", [0, top_locations[4,1:].tolist()], "Mach Speed:", mach_vectors[4], "Pressure:", pressure_vectors[4])

      else:
        results = []
        mach_vectorsa = []
        pressure_vectorsa = []

        for row in top_locations:
            row_results = []
            for value in row[1:]:
                row_results.append((value * 0.40, value * 0.60))
            results.append(row_results)

        # Unpack results if you need individual variables
        (ai1, ai2), (aj1, aj2), (ak1, ak2), (al1, al2) = results[0]
        (bi1, bi2), (bj1, bj2), (bk1, bk2), (bl1, bl2) = results[1]
        (ci1, ci2), (cj1, cj2), (ck1, ck2), (cl1, cl2) = results[2]
        (di1, di2), (dj1, dj2), (dk1, dk2), (dl1, dl2) = results[3]
        (ei1, ei2), (ej1, ej2), (ek1, ek2), (el1, el2) = results[4]

        smoothing = None
        while smoothing not in ["y","n"]:
          smoothing = input("Enable Smoothing? (y/n): ")
          if smoothing in ["y","n"]:
            if smoothing == "y":
              eta = np.array([0, ai1, ai2, aj1, aj2, ak1, ak2, al1, al2])
              eta.sort() #putting values in chronological order for smoother underbelly shape
              resulta = run_model(mach_inf=5, p_inf=5532, angles=eta)
              mach_vectorsa.append(resulta['mach'])
              pressure_vectorsa.append(resulta['pr'])

              etb = np.array([0, bi1, bi2, bj1, bj2, bk1, bk2, bl1, bl2])
              etb.sort()
              resultb = run_model(mach_inf=5, p_inf=5532, angles=etb)
              mach_vectorsa.append(resultb['mach'])
              pressure_vectorsa.append(resultb['pr'])

              etc = np.array([0, ci1, ci2, cj1, cj2, ck1, ck2, cl1, cl2])
              etc.sort()
              resultc = run_model(mach_inf=5, p_inf=5532, angles=etc)
              mach_vectorsa.append(resultc['mach'])
              pressure_vectorsa.append(resultc['pr'])

              etd = np.array([0, di1, di2, dj1, dj2, dk1, dk2, dl1, dl2])
              etd.sort()
              resultd = run_model(mach_inf=5, p_inf=5532, angles=etd)
              mach_vectorsa.append(resultd['mach'])
              pressure_vectorsa.append(resultd['pr'])

              ete = np.array([0, ei1, ei2, ej1, ej2, ek1, ek2, el1, el2])
              ete.sort()
              resulte = run_model(mach_inf=5, p_inf=5532, angles=ete)
              mach_vectorsa.append(resulte['mach'])
              pressure_vectorsa.append(resulte['pr'])

              print("Top 5:")
              print("1. Angles:", [eta.tolist()], "Mach Speed:", mach_vectorsa[0], "Pressure:", pressure_vectorsa[0])
              print("2. Angles:", [etb.tolist()], "Mach Speed:", mach_vectorsa[1], "Pressure:", pressure_vectorsa[1])
              print("3. Angles:", [etc.tolist()], "Mach Speed:", mach_vectorsa[2], "Pressure:", pressure_vectorsa[2])
              print("4. Angles:", [etd.tolist()], "Mach Speed:", mach_vectorsa[3], "Pressure:", pressure_vectorsa[3])
              print("5. Angles:", [ete.tolist()], "Mach Speed:", mach_vectorsa[4], "Pressure:", pressure_vectorsa[4])

            else:
              ta = np.array([0, ai1, ai2, aj1, aj2, ak1, ak2, al1, al2])
              resulta = run_model(mach_inf=5, p_inf=5532, angles=eta)
              mach_vectorsa.append(resulta['mach'])
              pressure_vectorsa.append(resulta['pr'])

              etb = np.array([0, bi1, bi2, bj1, bj2, bk1, bk2, bl1, bl2])
              resultb = run_model(mach_inf=5, p_inf=5532, angles=etb)
              mach_vectorsa.append(resultb['mach'])
              pressure_vectorsa.append(resultb['pr'])

              etc = np.array([0, ci1, ci2, cj1, cj2, ck1, ck2, cl1, cl2])
              resultc = run_model(mach_inf=5, p_inf=5532, angles=etc)
              mach_vectorsa.append(resultc['mach'])
              pressure_vectorsa.append(resultc['pr'])

              etd = np.array([0, di1, di2, dj1, dj2, dk1, dk2, dl1, dl2])
              resultd = run_model(mach_inf=5, p_inf=5532, angles=etd)
              mach_vectorsa.append(resultd['mach'])
              pressure_vectorsa.append(resultd['pr'])

              ete = np.array([0, ei1, ei2, ej1, ej2, ek1, ek2, el1, el2])
              resulte = run_model(mach_inf=5, p_inf=5532, angles=ete)
              mach_vectorsa.append(resulte['mach'])
              pressure_vectorsa.append(resulte['pr'])

              print("Top 5:")
              print("1. Angles:", [eta.tolist()], "Mach Speed:", mach_vectorsa[0], "Pressure:", pressure_vectorsa[0])
              print("2. Angles:", [etb.tolist()], "Mach Speed:", mach_vectorsa[1], "Pressure:", pressure_vectorsa[1])
              print("3. Angles:", [etc.tolist()], "Mach Speed:", mach_vectorsa[2], "Pressure:", pressure_vectorsa[2])
              print("4. Angles:", [etd.tolist()], "Mach Speed:", mach_vectorsa[3], "Pressure:", pressure_vectorsa[3])
              print("5. Angles:", [ete.tolist()], "Mach Speed:", mach_vectorsa[4], "Pressure:", pressure_vectorsa[4])

        else:
          exit()

    else:
      exit()

else:
  print("Mach Number is outside of range or cannot be read.")
  exit()



Epochs: 100
Epoch 1/100
200/200 [==============================] - 2s 4ms/step - loss: 13.2500 - mae: 2.6829 - val_loss: 7.8702 - val_mae: 2.1653
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 7.8540 - mae: 2.1684 - val_loss: 7.7596 - val_mae: 2.1518
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 7.7334 - mae: 2.1523 - val_loss: 7.6437 - val_mae: 2.1383
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 7.6039 - mae: 2.1356 - val_loss: 7.5072 - val_mae: 2.1201
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 7.4450 - mae: 2.1154 - val_loss: 7.3159 - val_mae: 2.0925
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 7.2329 - mae: 2.0872 - val_loss: 7.0740 - val_mae: 2.0597
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 6.9241 - mae: 2.0461 - val_loss: 6.6929 - val_mae: 2.0026
Epoch 8/100
200/200 [==============================] - 1